<a href="https://colab.research.google.com/github/taaha3244/RAG/blob/main/langfuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langfuse langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: pa

In [4]:
!pip -q install langchain openai tiktoken qdrant-client langchain_openai  pypdf langchainhub langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.6 MB/s eta 0:00:00


In [5]:
from langchain import hub
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import PromptTemplate

In [10]:
import os

# get keys for your project from https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] =
os.environ["LANGFUSE_SECRET_KEY"] =

os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # for EU data region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # for US data region

# your openai key
os.environ["OPENAI_API_KEY"] =

In [11]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler()

In [12]:
# Tests the SDK connection with the server
langfuse_handler.auth_check()

True

In [13]:
def Lex(query: str):
    """
    Takes a user query as input and returns a response using a Retrieval-Augmented Generation (RAG) flow
    incorporating langchain, Qdrant, and OpenAI.

    Args:
        query (str): The user's question to be answered.

    """
    try:
        # Setup
        qdrant_end = userdata.get('qdrant_end')
        qdrant_api_key = userdata.get('qdrant')
        openai_api_key=userdata.get('openai_api_key')

        embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
        qdrant_client = QdrantClient(url=qdrant_end, api_key=qdrant_api_key)
        qdrant = Qdrant(client=qdrant_client, collection_name="Lex-v1",
                        embeddings=embeddings_model)
        retriever = qdrant.as_retriever(search_kwargs={"k": 5})


        prompt=PromptTemplate(

       template="""
        # Your role
        You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer  from the docs to the questioner's needs from the documents you are given.


        # Instruction
        Your task is to answer the question  using the following pieces of retrieved context delimited by XML tags.

       <retrieved context>
        Retrieved Context:
        {context}
        </retrieved context>


        # Constraint
        1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question and provide the most appropriate answer.
        - Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
        2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
        3. Generate a concise, logical answer. When generating the answer, Do Not just list your selections, But rearrange them in context so that they become paragraphs with a natural flow.
       4. When you don't have retrieved context for the question or If you have a retrieved documents, but their content is irrelevant to the question, you should answer 'I can't find the answer to that question in the material I have'.
       5. Use five sentences maximum. Keep the answer concise but logical/natural/in-depth.
       6. At the end of the response provide metadata provided in the relevant docs , For example:"Metadata: page: 19, source: /content/OCR_RSCA/Analyse docs JVB + mails et convention FOOT INNOVATION.pdf'. Return Just the page and source


        # Question:
        {question}""",
        input_variables=["context","question"]
        )
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


        def format_docs(docs):
          formatted_docs = []
          for doc in docs:
           # Format the metadata into a string
            metadata_str = ', '.join(f"{key}: {value}" for key, value in doc.metadata.items())

          # Combine page content with its metadata
            doc_str = f"{doc.page_content}\nMetadata: {metadata_str}"

          # Append to the list of formatted documents
            formatted_docs.append(doc_str)

          # Join all formatted documents with double newlines
          return "\n\n".join(formatted_docs)


        rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
      )

        result = rag_chain.invoke(query,{"callbacks":[langfuse_handler]})
        return result
    except Exception as e:
      print(f"Error processing the query: {e}")
      return None


In [14]:
Lex("who is henrotay?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


'Henrotay is identified as Christophe Jean-Paul Luc, a Belgian national born on April 28, 1973, in Rocourt, Belgium. He is known to have resided in Monaco at 4, avenue des Ligures, 98000 MONACO. The retrieved information also mentions that Henrotay was active from April 28, 1973, and provides details about his legal residence in Neupré, Belgium. Additionally, there are references to his involvement in financial transactions that may require further investigation. \n\nMetadata: page: 2, source: /content/OCR_RSCA/Registre identifications.pdf'